# Week 2: Iterators, Generators, Decorators, Caching and Memoization

## Iterators

### What is iteration?

Iteration is the process of traversing a series of objects, handling each one in turn until there are no more objects to deal with.

For example, a for loop represents a type of iteration.

In python, we often iterate through a list:

```
numbers = [1,3,5,7,9]
for number in numbers:
    print(number)
```

### Iterable vs. iterator

* in python, an iterable object is one that implements an `__iter__` method
* in python, an iterator is an object that implements a `__next__` method

* iter() returns an iterator for an object
* next() returns the next element from an iterator (by invoking the underlying `__next__` method)
* usually the case that a class has both an ```__iter__``` and a ```__next__``` method

In [2]:
years = [1967, 1974, 1955, 2029]
years_iter = iter(years)
print(next(years_iter))
print(next(years_iter))
print(next(years_iter))
print(next(years_iter))
next(years_iter)

1967
1974
1955
2029


StopIteration: 

In [3]:
for year in years:
    print(year)

1967
1974
1955
2029


### Python's `for` loop is really a `while` loop with an iterator!
```
for x in some_iterable:
    do_something_with(x)
```

is really

```
iterator = iter(some_iterable)
while True:
    try:
        item = next(iterator)
    except StopIteration:
        break
    yield item
```

### Let's define a class that's iterable and that returns an iterator
(that is, one that returns an interator when ```__iter__``` is called)

In [6]:
class Decades:
    def __init__(self, years):
        self.years = years
        self.index = 0
    def __iter__(self):
        return self # note that self has a __next__ method, so it's an iterator
    def __next__(self):
        if self.index >= len(self.years):
            raise StopIteration
        retval = self.years[self.index] // 10 * 10
        self.index += 1
        return retval


In [ ]:
decades = Decades([1971,1982,2019])

In [ ]:
decades_iterator = iter(decades)

In [ ]:
next(decades_iterator)

## Generators

### Generators are similar to functions, so let's start there:

Let's look at a simple function that returns a list of n elements, each of which is a string containing "Hello"

In [ ]:
def Hello(n = 0):
    ret = []
    for i in range(n):
        ret.append("Hello")
    return ret

In [ ]:
hellos = Hello(5)

In [ ]:
for hello in hellos:
    print(hello)

## Our first generator


In [ ]:
def HelloGenerator(max = 0):
    x = 0
    while True:
        if x < max:
            yield "Hello"
            x = x+1
        else:
            break


In [ ]:
hellos = HelloGenerator(5)

In [ ]:
for hello in hellos:
    print(hello)

### Generators vs. Functions
* most important difference is that generators use the ```yield``` statement, whereas functions use ```return```
* ```yield``` returns a value, stops executing the code at that point and maintains state until it's called again
* when invoked, returns an object but doesn't start executing code
* implements ```__iter__``` and ```__next__``` automatically (hey, that's useful!)

### List comprehensions vs. generator expressions

In [7]:
# Initialize the list, in this case with a list of years
year_list = [2018, 1776, 2020, 1977, 1980, 2009, 2019]

# Find the decade corresponding to each of the years
decade_list = [x//10*10 for x in year_list]

In [8]:
# same thing with a generator
decade_generator = (x//10*10 for x in year_list)

In [9]:
max(decade_list)

2020

In [12]:
min(decade_generator)

ValueError: min() arg is an empty sequence

## Filtering

In the following code:

In [13]:
decade_generator_filtered = (x//10*10 for x in year_list if x > 1900)

the parentheses ('()') create a generator expression

In [15]:
max(decade_generator_filtered)

ValueError: max() arg is an empty sequence

## Memory size issues

In [ ]:
import sys

In [ ]:
sys.getsizeof(decade_list)

In [ ]:
sys.getsizeof(decade_generator)

In [ ]:
big_year_list = [x for x in range(1770,2020)]
big_decade_list = [x//10*10 for x in big_year_list]

In [ ]:
sys.getsizeof(big_decade_list)

In [ ]:
big_decade_generator = (x//10*10 for x in big_year_list)

In [ ]:
sys.getsizeof(big_decade_generator)

## A more complex example

In [ ]:
def lines_words_chars(text):
    yield ('lines',len(text.splitlines()))
    yield ('words',len(text.split()))
    yield ('characters',len(text))

In [ ]:
a = lines_words_chars("This is a text")

In [ ]:
next(a)

In [ ]:
next(a)

In [ ]:
next(a)

In [ ]:
next(a)

In [ ]:
# skip all non-lowercased letters (including punctuation)
# append 1 if lowercase letter is "o"
# append 0 if lowercase letter is not "o"
out = []
for i in "Hello. How Are You?":
    if i.islower():
        out.append(1 if i is "o" else 0)

In [ ]:
out

In [ ]:
# NOTE: this is not efficient because statistics.mean() will create a list from a generator
#       before proceeding with the calculation




from statistics import mean
out2 = mean(1 if char is 'o' else 0 for char in "Hello. How Are You?" if char.islower())
out2

## Let's take a look at some python code:

From https://github.com/python/cpython/blob/master/Lib/statistics.py


```
# === Measures of central tendency (averages) ===
def mean(data):
    """Return the sample arithmetic mean of data.
    >>> mean([1, 2, 3, 4, 4])
    2.8
    >>> from fractions import Fraction as F
    >>> mean([F(3, 7), F(1, 21), F(5, 3), F(1, 3)])
    Fraction(13, 21)
    >>> from decimal import Decimal as D
    >>> mean([D("0.5"), D("0.75"), D("0.625"), D("0.375")])
    Decimal('0.5625')
    If ``data`` is empty, StatisticsError will be raised.
    """
    if iter(data) is data:
        data = list(data)
    n = len(data)
    if n < 1:
        raise StatisticsError('mean requires at least one data point')
    T, total, count = _sum(data)
    assert count == n
    return _convert(total/n, T)
```

In [ ]:
data = [1,2,3]
idata = iter(data)

In [ ]:
data

In [ ]:
idata

In [ ]:
def dgen():
    yield 1
    yield 2
    yield 3

In [ ]:
dg = dgen()

In [ ]:
dgi = iter(dg)

In [ ]:
dg

In [ ]:
dgi

In [ ]:
if data is not idata:
    print(1)

In [ ]:
if iter(data) is data:
    print("yes")

In [ ]:
data

In [ ]:
iter(data)

## Memoization

A common way to teach memoization is to use Fibonacci numbers, defined as 

$ F_{0}=0,\quad F_{1}=1,$

and 

$ F_{n}=F_{n-1}+F_{n-2},$

for $n > 1$

Thus, the first few Fibonacci numbers are:
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55...

Here's an implementation of the code to calculate Fibonacci numbers:

### LEARNING CHECK POSSIBILITY: GET THEM TO WRITE A FIBONACCI FUNCTION

In [ ]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [ ]:
fibonacci(35)

### Important digression: Jupyter magic commands
* sometimes you'll see a line in a Jupyter notebook that starts with a '%'
* these are "magic" commands
* we'll deal with these in more detail in a later lecture, but for now we're going to introduce %timeit
* %timeit will tell you how much time a line takes to run
* %%timeit will tell you how much time a cell takes to run

In [ ]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [ ]:
%timeit fibonacci(32)

## Caching
* typically done with web browsers and web pages
* let's take a look at a simple caching example

## Memoization: a special form of caching
* caching is a more general approach: e.g. web pages
* memozation is caching of the output of a function given a specific set of parameters

## Memoization example: 

In [ ]:
def memoize(func):
    cache = dict()

    def memoized_func(*args):
        if args in cache:
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result

    return memoized_func

In [ ]:
memoized_fibonacci = memoize(fibonacci)

In [ ]:
%timeit memoized_fibonacci(32)

In [ ]:
%timeit memoized_fibonacci(30)

Note that the memoized version doesn't call the memoized verision when it recurses.

## Important Digression: Decorators
* recall example from above where one function (memoized_fibonacci) returned another function (fibonacci)
* this is a specific form of a more general approach called decorators
* let's take a look at the simplest form of a decorator, the null decorator, which does nothing

In [ ]:
def null_decorator(func):
    return func

In [ ]:
def hello():
    return "Hello"

In [ ]:
hello()

In [ ]:
decorated_hello = null_decorator(hello)

In [ ]:
decorated_hello()

### Now let's look at a slightly more complicated example that takes some function (assuming it returns a string) and wraps the output in ```<em>...</em>``` tags

In [ ]:
def emphasize(func):
    def wrapper():
        original_ret = func()
        modified_ret = "<em>" + original_ret + "</em>"
        return modified_ret
    return wrapper


In [ ]:
emphasized_hello = emphasize(hello)

In [ ]:
emphasized_hello()

### Using the @: wrapping functions simplified
* commonly referred to as "syntactic sugar", the @ command allows you to wrap a function with one line


In [ ]:
def emphasize(func):
    def wrapper():
        original_ret = func()
        modified_ret = "<em>" + original_ret + "</em>"
        return modified_ret
    return wrapper


In [ ]:
@emphasize
def hello():
    return "Hello"

In [ ]:
hello()

## A slightly more complicated example: decorating functions that take parameters
* let's say we have a function that returns "Hello" in some specified language:

In [ ]:
def multilingual_hello(lang = 'en'):
    lookup = {'en':'Hello','fr':'Bonjour'}
    return lookup[lang]

In [ ]:
multilingual_hello('en')

In [ ]:
multilingual_hello('fr')

## And now let's say we want to decorate that function with our emphasize wrapper:

In [ ]:
@emphasize
def multilingual_hello(lang = 'en'):
    lookup = {'en':'Hello','fr':'Bonjour'}
    return lookup[lang]

In [ ]:
multilingual_hello()

In [ ]:
multilingual_hello('fr')

### Uh oh... what just happened?
* our wrapper isn't set up to take any paramters, but our underlying function expects one (optional) one
* we can change our decorator to accommodate the optional paramter by using ```*args``` and ```**kwargs```:

In [ ]:
def emphasize_args(func):
    def wrapper(*args,**kwargs):
        original_ret = func(*args,**kwargs)
        modified_ret = "<em>" + original_ret + "</em>"
        return modified_ret
    return wrapper

In [ ]:
@emphasize_args
def multilingual_hello(lang = 'en'):
    lookup = {'en':'Hello','fr':'Bonjour'}
    return lookup[lang]

In [ ]:
multilingual_hello('fr')

## Ok, back to memoization
* but first, another digression: functools
* functools: https://docs.python.org/3/library/functools.html
* "Higher-order functions and operations on callable objects"
* of note, ``` @functools.lru_cache ```


In [ ]:
import functools

@functools.lru_cache(maxsize=128)
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

Or equivalently:

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

See also https://en.wikipedia.org/wiki/Cache_replacement_policies#Least_recently_used_(LRU) for LRU

In [ ]:
%timeit fibonacci(10)

In [ ]:
%timeit fibonacci(20)

In [ ]:
%timeit fibonacci(30)

In [ ]:
%timeit fibonacci(40)